In [1]:
import re
import requests as r
import time 
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px 
import plotly.io as pio 
pio.renderers.default = "browser"

In [2]:
def meanRetAn(data):             
    Result = 1
    
    for i in range(len(data.index)):
        Result *= (1+data.iloc[i,:])
        
    Result = Result**(1/float(len(data.index)/52))-1
     
    return(Result)

In [3]:
#getting etf tickers from etf in etf info
#etf_info = pd.read_csv("ETFs_info.csv")
yahoo_cat_df = pd.read_csv("yahoo_cat_df.csv",index_col="Ticker")
week_df_900 = pd.read_csv("WeeklyReturns.csv", index_col="Date")
etf_funda_df = pd.read_csv("etf_fundamental.csv", index_col="Unnamed: 0") # result from the fetcing code below
etf_funda_df = pd.merge(yahoo_cat_df,etf_funda_df, right_index=True,left_index=True, how="right")
etf_funda_df["pe"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["pcf"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["ps"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["pb"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["pe"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df["pcf"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df["ps"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df["pb"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df.drop(columns="Category", inplace=True)
print(len(etf_funda_df))
etf_funda_df = etf_funda_df.dropna()


1577


In [4]:
# dff = pd.DataFrame(mu_ga).rename(columns={"2016-08-05":"AnnRet"})
# dff["Annual Std"] = week_df_900.std(axis=0) * np.sqrt(52)


In [5]:
#etf_merged.dropna(inplace=True)
etf_merged = etf_funda_df.copy()

In [6]:
# # # fetching fundamentals from yahoo finance on each etf
# ticker = list(yahoo_cat_df.index)
# test = {}
# for i in tqdm(ticker):
#     test[i] = {}
#     lnk = f"https://finance.yahoo.com/quote/{i}/holdings?p={i}"
#     con = r.get(lnk)
#     soup = BeautifulSoup(con.text, "lxml")
#     try: 


#         test[i]["pe"] = soup.find('span', {"data-reactid":"131"}).get_text()
#         test[i]["pb"] = soup.find('span', {"data-reactid":"136"}).get_text()
#         test[i]["pcf"] = soup.find('span', {"data-reactid":"146"}).get_text()
#         test[i]["ps"] = soup.find('span', {"data-reactid":"141"}).get_text()
#     except:
#         test[i]["pe"] = np.nan
#         test[i]["pb"] = np.nan
#         test[i]["pcf"] = np.nan
#         test[i]["ps"] = np.nan
# a = pd.DataFrame(test).T
# a.to_csv("etf_fundamental.csv")

In [7]:
# Standardizing the features
cluster_stand = StandardScaler().fit_transform(etf_merged)
cluster_stand = pd.DataFrame(cluster_stand)

In [8]:
#Reducing dementionality of the data features to be explanined by 2 components for visualization and clustering 
X_embedded = TSNE(n_components=2, random_state=400).fit_transform(cluster_stand)
cluster_stand["dim1"] = X_embedded[:,0]
cluster_stand["dim2"] = X_embedded[:,1]
# cluster_stand = StandardScaler().fit_transform(etf_merged)
# cluster_stand = pd.DataFrame(etf_merged)


In [9]:
#finding the "optimal" cluster 
iini = []
clus = []
for n_clusters in range(2,14):
        clusterer = KMeans(n_clusters=n_clusters, random_state=10,n_init=20, algorithm="elkan",init='k-means++')
        cluster_labels = clusterer.fit_predict(cluster_stand)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(cluster_stand, cluster_labels)
        print("For n_clusters =", n_clusters,
            "The average silhouette_score is :", silhouette_avg)
        clus.append(n_clusters)
        iini.append(clusterer.inertia_)
fig = px.line(y=iini,x=clus)
fig.show()
        

For n_clusters = 2 The average silhouette_score is : 0.4437416238983516
For n_clusters = 3 The average silhouette_score is : 0.4768930952236609
For n_clusters = 4 The average silhouette_score is : 0.4663594218771804
For n_clusters = 5 The average silhouette_score is : 0.5120947574758284
For n_clusters = 6 The average silhouette_score is : 0.532826100163838
For n_clusters = 7 The average silhouette_score is : 0.5128123772992926
For n_clusters = 8 The average silhouette_score is : 0.4992627149746311
For n_clusters = 9 The average silhouette_score is : 0.4881527666984056
For n_clusters = 10 The average silhouette_score is : 0.49540928769275716
For n_clusters = 11 The average silhouette_score is : 0.48372961706526574
For n_clusters = 12 The average silhouette_score is : 0.4777085157850483
For n_clusters = 13 The average silhouette_score is : 0.4843768587342331


In [10]:

km = KMeans(
    n_clusters=6, init='k-means++',
    n_init=20, max_iter=1000, 
    tol=1e-04, random_state=10, algorithm="elkan"
)
clustered = km.fit_predict(cluster_stand)
cluster_stand["cluster"] = clustered
cluster_stand["dim1"] = X_embedded[:,0]
cluster_stand["dim2"] = X_embedded[:,1]
fig = px.scatter(cluster_stand,x="dim1", y="dim2", color="cluster")
fig.show()

In [11]:
week_df_900 = pd.merge(week_df_900.T,yahoo_cat_df, right_index=True, left_index= True, how="inner")
week_df_900 = week_df_900.iloc[:,:-1].T

In [12]:
etf_merged["cluster"] = clustered
list1_as_set = set(week_df_900.columns)
intersection = list1_as_set.intersection(list(etf_funda_df.index))
intersection_as_list = list(intersection)
fund_min = etf_merged.loc[intersection_as_list,:]
week_df_900 = week_df_900[intersection_as_list]


In [13]:
dfff = pd.DataFrame(index=week_df_900.index)
dff_dict = {}
for i in fund_min["cluster"].unique():
    tickers = list(fund_min[fund_min["cluster"]==i].index)
    n_assets = len(tickers)
    portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
    port_weekly_return = week_df_900[tickers].mul(portfolio_weights_ew,axis=1).sum(axis=1)
    dfff[i] = port_weekly_return
    
cumsum = dfff.cumsum(axis=0)

In [14]:
cluster_sort = pd.DataFrame(dfff.std(axis=0)*np.sqrt(52)).reset_index().sort_values(by=[0], ascending=False)["index"]


In [15]:
pd.DataFrame(dfff.std(axis=0)*np.sqrt(52)).reset_index().sort_values(by=[0], ascending=False)

,index,0
2,4,0.486448
4,0,0.383660
3,5,0.227008
5,3,0.191325
0,2,0.185675
1,1,0.179898


In [16]:
cumsum["SPY"]= week_df_900["SPY"].cumsum(axis=0)


In [17]:
cumsum

,2,1,4,5,0,3,SPY
2016-08-05,0.000060,-0.001394,0.014341,0.008903,-0.015368,0.004237,0.004882
2016-08-12,0.013185,0.007512,0.033149,0.015900,-0.030839,0.009462,0.006165
2016-08-19,0.014469,0.008427,0.037784,0.025045,-0.039478,0.010276,0.006532
2016-08-26,0.004608,0.000307,0.005164,0.016371,-0.014646,0.004384,0.000812
2016-09-02,0.016023,0.010510,0.031707,0.023125,-0.035351,0.011258,0.005782
...,...,...,...,...,...,...,...
2020-03-20,-0.152343,-0.008503,-0.166134,-0.285710,-0.085414,0.182712,0.177057
2020-03-27,-0.051788,0.097563,0.077140,-0.193538,-0.321657,0.290444,0.284662
2020-04-03,-0.085751,0.067011,-0.001871,-0.219320,-0.285898,0.264435,0.264024
2020-04-17,0.034837,0.197809,0.368406,-0.054231,-0.547107,0.420024,0.418946


In [18]:

plot = px.line(cumsum, x = cumsum.index, y = cumsum.columns) 
plot.show()

In [19]:
for i in cluster_sort:
    print(i)
    if "SPY" in etf_merged[ etf_merged["cluster"]==i].index: 
        print("SPY in:",i)
    print("Number of Assets:",len(etf_merged[ etf_merged["cluster"]==i]))
    print("Price/Sales",etf_merged[ etf_merged["cluster"]==i]["ps"].mean()) 
    print("Price/Book",etf_merged[ etf_merged["cluster"]==i]["pb"].mean()) 
    print("Price/Casflow",etf_merged[ etf_merged["cluster"]==i]["pcf"].mean()) 
    print("Price/Earnings",etf_merged[ etf_merged["cluster"]==i]["pe"].mean()) 
    avg = etf_merged[ etf_merged["cluster"]==i]["ps"].mean()+etf_merged[ etf_merged["cluster"]==i]["pb"].mean()+etf_merged[ etf_merged["cluster"]==i]["pcf"].mean()+etf_merged[          etf_merged["cluster"]==i]["pe"].mean()
    print(avg/4)
    print(20*"-")

4
Number of Assets: 75
Price/Sales -5.0
Price/Book -5.0
Price/Casflow -5.0
Price/Earnings -5.0
-5.0
--------------------
0
Number of Assets: 65
Price/Sales -1.0
Price/Book -1.0
Price/Casflow -1.0
Price/Earnings -1.0
-1.0
--------------------
5
Number of Assets: 161
Price/Sales 1.9332919254658385
Price/Book 1.8878260869565218
Price/Casflow 2.8816770186335408
Price/Earnings 4.156832298136646
2.7149068322981367
--------------------
3
SPY in: 3
Number of Assets: 432
Price/Sales 3.550694444444445
Price/Book 5.695324074074074
Price/Casflow 18.94199074074074
Price/Earnings 29.77212962962963
14.490034722222223
--------------------
2
Number of Assets: 389
Price/Sales 1.0327506426735218
Price/Book 1.567146529562982
Price/Casflow 7.349588688946015
Price/Earnings 16.474884318766065
6.6060925449871455
--------------------
1
Number of Assets: 438
Price/Sales 1.7493150684931509
Price/Book 2.5243835616438353
Price/Casflow 11.68566210045662
Price/Earnings 22.511050228310502
9.617602739726028
----------

In [20]:
test = pd.DataFrame(dfff.std(axis=0)*np.sqrt(52)).reset_index().sort_values(by=[0], ascending=False)
annn = pd.DataFrame(meanRetAn(dfff)).reset_index().rename(columns={"2016-08-05":"AnnRet"})
dingo = pd.merge(test,annn,on="index",how="inner")

In [21]:
yahoo_df = pd.read_csv("yahoo_cat_df.csv", index_col="Ticker")

In [22]:
com_df = pd.merge(etf_merged,yahoo_cat_df,right_index=True,left_index=True,how="inner")


In [23]:
for i in cluster_sort:
    print(i)
    print(len(com_df[com_df["cluster"]==i]), sum(com_df[com_df["cluster"]==i]["Category"].value_counts()))
    print(com_df[com_df["cluster"]==i]["Category"].value_counts())
    print(30*"-")

4
75 75
Trading--Leveraged Equity    75
Name: Category, dtype: int64
------------------------------
0
65 65
Trading--Inverse Equity    65
Name: Category, dtype: int64
------------------------------
5
161 134
Financial                     30
Equity Energy                 21
Energy Limited Partnership    14
Health                        11
Miscellaneous Sector           8
Miscellaneous Region           6
Technology                     6
Large Growth                   5
World Stock                    4
Consumer Cyclical              4
Large Blend                    3
Europe Stock                   3
Large Value                    3
Option Writing                 2
Bear Market                    2
Natural Resources              2
Small Growth                   2
Equity Precious Metals         2
Foreign Large Value            1
Pacific/Asia ex-Japan Stk      1
Diversified Emerging Mkts      1
China Region                   1
Small Blend                    1
Nontraditional Bond            1


In [24]:
funda_cluster = {}
for i,x in zip(cluster_sort,["Market Leveraged ETF","Market Inverse ETF","Low Fundamental ETF’s (Energy, Health,Finance)","High Fundamental Broad Market","Low Fundamental Broad Market","Mid Fundamental Broad Market"]):
    funda_cluster[i] = x

In [25]:
funda_cluster

{4: 'Market Leveraged ETF',
 0: 'Market Inverse ETF',
 5: 'Low Fundamental ETF’s (Energy, Health,Finance)',
 3: 'High Fundamental Broad Market',
 2: 'Low Fundamentals Broad Market',
 1: 'Mid Fundamentals Broad Market'}

In [29]:
com_df["Fundamental Cluster"] = com_df["cluster"].apply(lambda x: funda_cluster[x])

In [33]:
com_df.to_csv("Fundamental_Clustering.csv", index_label="Ticker")

In [27]:
fundamental_df_cluster = com_df.copy()

In [35]:
pd.read_csv("Fundamental_Clustering.csv", index_col="Ticker")

,pe,pb,pcf,ps,cluster,Category,Fundamental Cluster
Ticker,,,,,,,
SPY,27.93,4.00,16.31,2.74,3,Large Blend,High Fundamental Broad Market
IVV,27.16,3.81,16.08,2.62,3,Large Blend,High Fundamental Broad Market
VTI,27.49,3.81,15.92,2.58,3,Large Blend,High Fundamental Broad Market
VOO,27.94,4.00,16.32,2.75,3,Large Blend,High Fundamental Broad Market
QQQ,34.42,7.93,21.00,4.64,3,Large Growth,High Fundamental Broad Market
...,...,...,...,...,...,...,...
DMAY,27.94,4.00,16.31,2.74,3,NaN,High Fundamental Broad Market
NJUL,34.65,7.86,21.31,4.77,3,Large Blend,High Fundamental Broad Market
BMAY,27.17,3.81,16.08,2.62,3,Large Blend,High Fundamental Broad Market
